In [25]:
import pandas as pd

In [26]:
data = pd.read_excel(
    "/Users/joaoalmeida/Downloads/Tabela-MCDT-Convencionados-2024_05.09.2024.xlsx",
    sheet_name="A-Análises",
)

/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'A-Análises'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'L-Neurofisiologia'!$A:$E.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'N-Consultas'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/opt/homebrew/Caskroom/miniforge/base/envs/py3/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'O-Psicologia'!$A:$D.
  warn(f"Print area cannot be set to Defined name: {d

In [27]:
data.columns = ["cod_sns", "cod_conv", "Nomenclatura", "preco", "taxa"]
data.iloc[3:, :]


,cod_sns,cod_conv,Nomenclatura,preco,taxa
3,NaN,NaN,BIOQUÍMICA,NaN,NaN
4,21175,684.0,"17 alfa-hidroxiprogesterona, s",7.4,1.4
5,21443,1025.6,"17-cetosteróides, u",6.9,1.5
6,22187,1050.7,"17-hidroxicorticosteróides (17-OHCS), u",6.31,1.5
7,22511,569.0,"5'-nucleotidase, s",3.1,0.8
...,...,...,...,...,...
317,55010,079.5,Tipagem AB0 e Rh (D),5.27,1.6
318,NaN,NaN,NaN,NaN,NaN
319,"a) A entrada em vigor deste código, carece de ...",NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN


In [ ]:
def create_codesystem_mcdts_fsh(row):
    # print(row)
    if pd.isna(row["cod_sns"]):
        print(row["Nomenclatura"])
    else:
        code = row["cod_sns"]
        code_conv = row["cod_conv"]
        name = row["Nomenclatura"]
        preco = row["preco"]
        taxa = row["taxa"]

        file.write(
            "* #" + str(code) + '\n    "' + str(name) + '"\n    "' + str(name) + '"\n'
        )
        # print(preco)
        if not pd.isna(preco):
            try:
                file.write(
                    "  * ^property.code = #preco\n  * ^property.valueDecimal = "
                    + str(round(preco, 2))
                    + "\n  * ^property.code = #taxa\n  * ^property.valueDecimal = "
                    + str(round(taxa, 2))
                    + "\n"
                )
            except:
                print("ERRROR", preco)


with open("example.txt", "w") as file:
    row_sums = data.iloc[3:, :].apply(
        create_codesystem_mcdts_fsh, axis=1
    )  # Axis 1 applies the function row-wise


BIOQUÍMICA
ERRROR a)
ERRROR a)
ERRROR a)
Equilíbrio ácido-base (pH, pCO2, pO2, SatO2, CO2, ...), s (ver código 21789 - Gasimetria…)
ERRROR a)
ERRROR a)
Morfina (ver código 21775 - Drogas de abuso, pesquisa, cada, s/u)
ERRROR a)
ERRROR a)
ERRROR a)
nan
HEMATOLOGIA
ERRROR a)
nan
HEMOSTASE
ERRROR a)
nan
IMUNOLOGIA
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
ERRROR a)
CITOMETRIA DE FLUXO
O estudo por citometria de fluxo é um estudo de carácter interpretativo, que pode exigir uma abordagem sequencial. Assim, em alguns casos, foi contemplada a possibilidade de dois tipos de estudo que podem ser efetuados sequencialmente na mesma amostra em caso de necessidade: um primeiro estudo, designado por estudo inicial e um segundo, designado por estudo complementar.
Anticorpos, pesquisa em células e em soro
Doenças linfoproliferativas, imunofenotipagem
Imunodeficiências e doenças autoimunes, caracterização de distúrbios da imunidade
nan
MICROBIOLOGIA
Serologia
Os códigos que não explic